# Sloth & ENVy | PyData New York 2019
> 'Don't show up to this talk, it's going to be stupid. You won't learn anything, and if you do learn something, I promise it will be dumb.' - Diego Torres Quintanilla, PyData NYC Executive Chair

A thought question, "Did you know that the software you use and rely upon everyday was written and designed just by some person?" and ,"Did you know that the software you use and rely upon everyday has code you can read and fully understand?" and ,"Did you know that the software you use and rely upon everyday is probably written in C or relies on some core libraries that were written in C, in which case the System V Application Binary Interface (or 'ABI') for AMD64 specifies that the entry point can have one of three possible signatures as follows

    * int main()
    * int main(int, char*[])
    * int main(int, char*[], char*[])
    
The last can be extended as `int main(int argc, char *argv[], char* envp[]) {}`. envp is a pointer to the environment

What do I mean by an environment? I mean the thing when you type `env` in terminal.

In [ ]:
!env

LESSOPEN=| /usr/bin/lesspipe %s
CONDA_PROMPT_MODIFIER=(torch) 
USER=kushaj
LANGUAGE=en_IN:en
TEXTDOMAIN=im-config
CONDA_MKL_INTERFACE_LAYER_BACKUP=
XDG_SEAT=seat0
MPLBACKEND=module://ipykernel.pylab.backend_inline
SSH_AGENT_PID=1741
XDG_SESSION_TYPE=x11
LD_LIBRARY_PATH=/usr/local/cuda-10.1/lib64
SHLVL=1
CONDA_SHLVL=2
OLDPWD=/home/kushaj/Desktop/Temp
QT4_IM_MODULE=xim
HOME=/home/kushaj
DESKTOP_SESSION=ubuntu
GIO_LAUNCHED_DESKTOP_FILE=/etc/xdg/autostart/org.gnome.SettingsDaemon.MediaKeys.desktop
GNOME_SHELL_SESSION_MODE=ubuntu
GTK_MODULES=gail:atk-bridge
PAGER=cat
PS1=(torch) \u@\h: 
DBUS_SESSION_BUS_ADDRESS=unix:path=/run/user/1000/bus
COLORTERM=truecolor
GIO_LAUNCHED_DESKTOP_FILE_PID=1946
_CE_M=
TERMINATOR_DBUS_NAME=net.tenshu.Terminator20x1a6021154d881c
MANDATORY_PATH=/usr/share/gconf/ubuntu.mandatory.path
IM_CONFIG_PHASE=2
INFOPATH=/usr/local/texlive/2019/texmf-dist/doc/info:
LOGNAME=kushaj
GTK_IM_MODULE=ibus
DESKTOP_AUTOSTART_ID=10fe220e7fc4becbab158588059531334000000016450007
_=/ho

If you want to get a more closer view you can do something as below. It is just a giant set of null-terminated strings where each string has some equal sign in it and the first part of the string is key and the other part is the value. It is some key-value that each program has access to.

In [ ]:
!less /proc/self/environ

:N=| /usr/bin/lesspipe %s^@CONDA_PROMPT_MODIFIER=(torch) ^@USER=kushaj^@LANGUAGE=en_IN:en^@TEXTDOMAIN=im-config^@CONDA_MKL_INTERFACE_LAYER_BACKUP=^@XDG_SEAT=seat0^@MPLBACKEND=module://ipykernel.pylab.backend_inline^@SSH_AGENT_PID=1741^@XDG_SESSION_TYPE=x11^@LD_LIBRARY_PATH=/usr/local/cuda-10.1/lib64^@SHLVL=1^@CONDA_SHLVL=2^@OLDPWD=/home/kushaj/Desktop/Temp^@QT4_IM_MODULE=xim^@HOME=/home/kushaj^@DESKTOP_SESSION=ubuntu^@GIO_LAUNCHED_DESKTOP_FILE=/etc/xdg/autostart/org.gnome.SettingsDaemon.MediaKeys.desktop^@GNOME_SHELL_SESSION_MODE=ubuntu^@GTK_MODULES=gail:atk-bridge^@PAGER=cat^@PS1=(torch) \u@\h: ^@DBUS_SESSION_BUS_ADDRESS=unix:path=/run/user/1000/bus^@COLORTERM=truecolor^@GIO_LAUNCHED_DESKTOP_FILE_PID=1946^@_CE_M=^@TERMINATOR_DBUS_NAME=net.tenshu.Terminator20x1a6021154d881c^@MANDATORY_PATH=/usr/share/gconf/ubuntu.mandatory.path^@IM_CONFIG_PHASE=2^@INFOPATH=/usr/local/texlive/2019/texmf-dist/doc/info:^@LOGNAME=kushaj^@GTK_IM_MODULE=ibus^@DESKTOP_AUTOSTART_ID=10fe220e7fc4becbab1585880595

In [ ]:
!echo $PATH

/home/kushaj/gems/bin:/usr/local/texlive/2019/bin/x86_64-linux:/usr/local/cuda-10.1/bin:/usr/local/cuda-10.1/NsightCompute-2019.3:/home/kushaj/anaconda3/envs/torch/bin:/home/kushaj/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/kushaj/hugo/v56


The path identifies what happens when you type the name of a binary that gets run when you want to run something.

When you look at your path it is just a colon(:) separated set of directories. What if you have a directory with a colon in it's name that you want to add in your path? The answer is you can't.

Sometimes you would have wondered which python you are using.

In [ ]:
!which python

/home/kushaj/anaconda3/envs/torch/bin/python


In [ ]:
!which which

/usr/bin/which


In [ ]:
!which /usr/bin/which

/usr/bin/which


You should prefer `python3.8 -m pip install` instead of `pip install`. The reason is that this ensures you are using the correct pip. In the first case you pick the python and that looks at the path so see that you are using a conda installed pip and thus uses the pip in conda. But you cannot use `python -m conda install`.

Now if I look at the C file.
```c
#include<stdio.h>
#include<stdlib.h>

int main(int argc, char *argv[], char* envp[]){}
```

Now where does this envp come from and what is its purpose. So we look at the binary and we disassemble. We know that every C program starts with `<_start>` and it runs a bunch of stuff.

In [ ]:
!objdump -S a.out | grep -A 15 '<_start>'

00000000000004f0 <_start>:
 4f0:	31 ed                	xor    %ebp,%ebp
 4f2:	49 89 d1             	mov    %rdx,%r9
 4f5:	5e                   	pop    %rsi
 4f6:	48 89 e2             	mov    %rsp,%rdx
 4f9:	48 83 e4 f0          	and    $0xfffffffffffffff0,%rsp
 4fd:	50                   	push   %rax
 4fe:	54                   	push   %rsp
 4ff:	4c 8d 05 7a 01 00 00 	lea    0x17a(%rip),%r8        # 680 <__libc_csu_fini>
 506:	48 8d 0d 03 01 00 00 	lea    0x103(%rip),%rcx        # 610 <__libc_csu_init>
 50d:	48 8d 3d e6 00 00 00 	lea    0xe6(%rip),%rdi        # 5fa <main>
 514:	ff 15 c6 0a 20 00    	callq  *0x200ac6(%rip)        # 200fe0 <__libc_start_main@GLIBC_2.2.5>
 51a:	f4                   	hlt    
 51b:	0f 1f 44 00 00       	nopl   0x0(%rax,%rax,1)

0000000000000520 <deregister_tm_clones>:


One interesting thing is that the int main that you write which can have three possible signatures is not what actually runs when you launch the binary. What actually runs is libc_start_main which then calls your main, which is the reason why you can have three different signatures in your C file with different number of arguments. Now if you dig into the source code of libc, you can see that there is a main function that is called and in that function there is an environ variable that stores the value of env. And this is the reason why you can use environ in python.

In [ ]:
from os import environ, getenv, putenv

In [ ]:
# Shows all the environment variables in your environment
environ

environ{'CLUTTER_IM_MODULE': 'xim',
        'CONDA_SHLVL': '2',
        'LD_LIBRARY_PATH': '/usr/local/cuda-10.1/lib64',
        'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.Z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.zst=01;31:*.tzst=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.war=01;31:*.ear=01;31:*.sar=01;31:*.rar=01;31:*.alz=01;31:*.ace=01;31:*.zoo=01;31:*.cpio=01;31:*.7z=01;31:*.rz=01;31:*.cab=01;31:*.wim=01;31:*.swm=01;31:*.dwm=01;31:*.esd=01;31:*.jpg=01;35:*.jpeg=01;35:*.mjpg=01;35:*.mjpeg=01;35:*.gif=01;35:*.bmp=01;35:*.pbm=01;35:*.pgm=01;35:*.ppm=01;35:*.tga=01;35:*.xbm=01;35:*.xp

The next question we ask, is we know that there are no data structures in C. So when we looked at environ using `less /proc/self/environ`, we noticed that it is a string. Now we know that in python we have a dict which has constant time loopup which string has a linear time loopup. So wouldn't it be faster to implement environ using python dict. To check this we write a simple script to check the times, as we are mostly told that the most important thing is your Big O complexity. And we see that C program is faster. But as we reach to 100,000 environment variables the time complexity of linear starts catching up and we see that C program becomes slower in that case.

In modern python we are told that getters and setters are not pythonic. But the interesting things is that with python 3.7 in sys module we have getters and setters. 

What is the difference between `from math import log` and `import math; math.log(x)`. Which of these is right? The difference is very subtle and it ties back to the environment store. The difference is that one of these is early binding and the other is late binding. In other words when you say `from math import log` it is the same as doing below.

In [ ]:
import math
log = math.log
log(1)

But the meaningful difference is are you capturing the value of that function as of the line when you did the import or you are capturing the value of the function whenever you are calling that function out of that module. Are you getting the most recent value or the least recent value.

One of the general guidelines with environment variables is that don't change them over the course of the program because you don't know when some program might call getter or setter. So set the environment variables at the start and don't change them in your main process.